In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [3]:
import random

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

39 documents
7 classes ['answer', 'goodbye', 'greeting', 'noanswer', 'options', 'surprise', 'thanks']
53 unique lemmatized words ["'s", ',', 'a', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'could', 'day', 'do', 'fact', 'for', 'fsdh', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'is', 'later', 'lookup', 'me', 'next', 'nice', 'offered', 'provide', 'random', 'see', 'sjdfhk', 'support', 'surprise', 'tell', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'wa', 'what', 'when', 'where', 'who', 'you']


In [7]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [8]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")
print("X = ", train_x)
print()
print()
print("Y = ", train_y)


Training data created
X =  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0,

In [9]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [10]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [11]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=400, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

Epoch 1/400
8/8 [==============================] - 0s 6ms/step - loss: 1.9819 - accuracy: 0.1282
Epoch 2/400
8/8 [==============================] - 0s 2ms/step - loss: 1.9253 - accuracy: 0.2051
Epoch 3/400
8/8 [==============================] - 0s 2ms/step - loss: 1.8923 - accuracy: 0.2051
Epoch 4/400
8/8 [==============================] - 0s 2ms/step - loss: 1.8076 - accuracy: 0.3333
Epoch 5/400
8/8 [==============================] - 0s 3ms/step - loss: 1.7955 - accuracy: 0.2564
Epoch 6/400
8/8 [==============================] - 0s 2ms/step - loss: 1.7620 - accuracy: 0.3590
Epoch 7/400
8/8 [==============================] - 0s 1ms/step - loss: 1.6879 - accuracy: 0.3590
Epoch 8/400
8/8 [==============================] - 0s 3ms/step - loss: 1.6212 - accuracy: 0.3077
Epoch 9/400
8/8 [==============================] - 0s 2ms/step - loss: 1.5503 - accuracy: 0.3846
Epoch 10/400
8/8 [==============================] - 0s 2ms/step - loss: 1.5072 - accuracy: 0.4872
Epoch 11/400
8/8 [===========

8/8 [==============================] - 0s 2ms/step - loss: 0.0596 - accuracy: 1.0000
Epoch 85/400
8/8 [==============================] - 0s 2ms/step - loss: 0.0815 - accuracy: 1.0000
Epoch 86/400
8/8 [==============================] - 0s 2ms/step - loss: 0.0978 - accuracy: 1.0000
Epoch 87/400
8/8 [==============================] - 0s 2ms/step - loss: 0.0991 - accuracy: 0.9487
Epoch 88/400
8/8 [==============================] - 0s 2ms/step - loss: 0.0869 - accuracy: 1.0000
Epoch 89/400
8/8 [==============================] - 0s 2ms/step - loss: 0.0779 - accuracy: 1.0000
Epoch 90/400
8/8 [==============================] - 0s 1ms/step - loss: 0.1230 - accuracy: 0.9487
Epoch 91/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0999 - accuracy: 0.9487
Epoch 92/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0748 - accuracy: 1.0000
Epoch 93/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0853 - accuracy: 1.0000
Epoch 94/400
8/8 [===============

8/8 [==============================] - 0s 2ms/step - loss: 0.0463 - accuracy: 1.0000
Epoch 168/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0490 - accuracy: 0.9744
Epoch 169/400
8/8 [==============================] - 0s 2ms/step - loss: 0.0494 - accuracy: 0.9744
Epoch 170/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0218 - accuracy: 1.0000
Epoch 171/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0379 - accuracy: 0.9744
Epoch 172/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0136 - accuracy: 1.0000
Epoch 173/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0844 - accuracy: 1.0000
Epoch 174/400
8/8 [==============================] - 0s 991us/step - loss: 0.0368 - accuracy: 0.9744
Epoch 175/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0210 - accuracy: 1.0000
Epoch 176/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0322 - accuracy: 1.0000
Epoch 177/400
8/8 [===

8/8 [==============================] - 0s 1ms/step - loss: 0.0490 - accuracy: 0.9744
Epoch 250/400
8/8 [==============================] - 0s 2ms/step - loss: 0.0111 - accuracy: 1.0000
Epoch 251/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0110 - accuracy: 1.0000
Epoch 252/400
8/8 [==============================] - 0s 2ms/step - loss: 0.0307 - accuracy: 1.0000
Epoch 253/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0533 - accuracy: 1.0000
Epoch 254/400
8/8 [==============================] - 0s 2ms/step - loss: 0.0078 - accuracy: 1.0000
Epoch 255/400
8/8 [==============================] - 0s 2ms/step - loss: 0.0569 - accuracy: 1.0000
Epoch 256/400
8/8 [==============================] - 0s 2ms/step - loss: 0.0282 - accuracy: 1.0000
Epoch 257/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 258/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0558 - accuracy: 0.9744
Epoch 259/400
8/8 [=====

8/8 [==============================] - 0s 1ms/step - loss: 0.0201 - accuracy: 1.0000
Epoch 332/400
8/8 [==============================] - 0s 2ms/step - loss: 0.0452 - accuracy: 0.9744
Epoch 333/400
8/8 [==============================] - 0s 2ms/step - loss: 0.0068 - accuracy: 1.0000
Epoch 334/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0117 - accuracy: 1.0000
Epoch 335/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0354 - accuracy: 1.0000
Epoch 336/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0418 - accuracy: 0.9744
Epoch 337/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0195 - accuracy: 1.0000
Epoch 338/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0107 - accuracy: 1.0000
Epoch 339/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0146 - accuracy: 1.0000
Epoch 340/400
8/8 [==============================] - 0s 1ms/step - loss: 0.0346 - accuracy: 1.0000
Epoch 341/400
8/8 [=====